In [1]:
%load_ext autoreload
%autoreload 2

# Import dependencies

In [34]:
import os

from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.runnables import chain

from IPython.display import display, Markdown, Latex

from typing import List

# Configure Google credentials
- **NOTE**: Remember change the `GOOGLE_APPLICATION_CREDENTIALS` to the path of your own Google credentials file.

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    "/mnt/Exdisk/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

# Document

In [4]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

# Loading documents

In [5]:
file_path = "./../../../data/sample.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

In [6]:
print(len(docs))

360


In [7]:
print(f"{docs[1].page_content[:200]}\n")
print(docs[1].metadata)

2
VKS
VKS  VNGCloud Kubernetes Service) is a managed service on VNGCloud that helps
you simplify the deployment and management of container-based applications.
Kubernetes is an open-source platform de

{'source': './../../../data/sample.pdf', 'page': 1}


# Spliting

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [9]:
len(all_splits)

535

In [10]:
all_splits[100]

Document(metadata={'source': './../../../data/sample.pdf', 'page': 67, 'start_index': 0}, page_content='68')

# Embeddings

In [11]:
embeddings = VertexAIEmbeddings(model="text-embedding-004")

vector_1 = embeddings.embed_query(all_splits[100].page_content)
vector_2 = embeddings.embed_query(all_splits[101].page_content)

In [12]:
assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:20])

Generated vectors of length 768

[0.012313091196119785, -0.0286110732704401, -0.04371386021375656, -0.007997739128768444, 0.030984440818428993, 0.03765463829040527, 0.005473416298627853, -0.004623760934919119, 0.015282704494893551, 0.007988318800926208, 0.005866541992872953, 0.04319928586483002, 0.057796817272901535, -0.035963620990514755, -0.03859233856201172, -0.07828214019536972, -0.04263092204928398, -0.04011708125472069, -0.06994833797216415, -0.06528584659099579]


# Vector stores

In [13]:
vector_store = InMemoryVectorStore(embeddings)

In [14]:
ids = vector_store.add_documents(documents=all_splits)

# Usage

In [15]:
results = vector_store.similarity_search("What is VKS private cluster?")

print(results[0])

page_content='Private Cluster feature helps your K8S cluster to be as secure as possible, all
connections are completely private from the connection between the nodes to
the control plane, the connection from the client to the control plane, or the
connection from the nodes to other products and services in VNG Cloud such
as: vStorage, vCR, vMonitor, VNGCloud APIs,... Private Cluster is the ideal
choice for services that require strict access control, ensuring compliance
with regulations on security and data privacy. For details on the two operating
models of Cluster, you can refer to here and refer to the steps to create a
private Cluster here .
VKS  VNGCloud Kubernetes Service) introduces the latest update for VKS, bringing
numerous new improvements for users. Here are the details of the update:
Aug 28, 2024
Aug 26, 2024' metadata={'source': './../../../data/sample.pdf', 'page': 11, 'start_index': 833}


In [16]:
len(results)

4

In [17]:
for res in results:
    print(res, end=f"\n{'_'*100}\n")

page_content='Private Cluster feature helps your K8S cluster to be as secure as possible, all
connections are completely private from the connection between the nodes to
the control plane, the connection from the client to the control plane, or the
connection from the nodes to other products and services in VNG Cloud such
as: vStorage, vCR, vMonitor, VNGCloud APIs,... Private Cluster is the ideal
choice for services that require strict access control, ensuring compliance
with regulations on security and data privacy. For details on the two operating
models of Cluster, you can refer to here and refer to the steps to create a
private Cluster here .
VKS  VNGCloud Kubernetes Service) introduces the latest update for VKS, bringing
numerous new improvements for users. Here are the details of the update:
Aug 28, 2024
Aug 26, 2024' metadata={'source': './../../../data/sample.pdf', 'page': 11, 'start_index': 833}
__________________________________________________________________________________

In [18]:
results[0]

Document(id='5cecfd8e-53fc-4078-8c5d-1d2d4553cd3b', metadata={'source': './../../../data/sample.pdf', 'page': 11, 'start_index': 833}, page_content='Private Cluster feature helps your K8S cluster to be as secure as possible, all\nconnections are completely private from the connection between the nodes to\nthe control plane, the connection from the client to the control plane, or the\nconnection from the nodes to other products and services in VNG Cloud such\nas: vStorage, vCR, vMonitor, VNGCloud APIs,... Private Cluster is the ideal\nchoice for services that require strict access control, ensuring compliance\nwith regulations on security and data privacy. For details on the two operating\nmodels of Cluster, you can refer to here and refer to the steps to create a\nprivate Cluster here .\nVKS \x00VNGCloud Kubernetes Service) introduces the latest update for VKS, bringing\nnumerous new improvements for users. Here are the details of the update:\nAug 28, 2024\nAug 26, 2024')

# Async query

In [21]:
results = await vector_store.asimilarity_search(
    "How to expose service using LoadBalancer in VKS?"
)

In [22]:
print(results[0])

page_content='315
• In case of migrating to VKS and still using vcontainer-nginx-ingress-controller, it
is necessary to change the Service type to LoadBalancer
velero restore create --item-operation-timeout 1m --from-backup vcontaine
velero restore create --item-operation-timeout 1m --from-backup vcontaine
velero restore create --item-operation-timeout 1m --from-backup vcontaine
kubectl patch service -n kube-system vcontainer-ingress-nginx-controller' metadata={'source': './../../../data/sample.pdf', 'page': 314, 'start_index': 0}


In [23]:
for res in results:
    print(res, end=f"\n{'_'*100}\n")

page_content='315
• In case of migrating to VKS and still using vcontainer-nginx-ingress-controller, it
is necessary to change the Service type to LoadBalancer
velero restore create --item-operation-timeout 1m --from-backup vcontaine
velero restore create --item-operation-timeout 1m --from-backup vcontaine
velero restore create --item-operation-timeout 1m --from-backup vcontaine
kubectl patch service -n kube-system vcontainer-ingress-nginx-controller' metadata={'source': './../../../data/sample.pdf', 'page': 314, 'start_index': 0}
____________________________________________________________________________________________________
page_content='Ingress Controller: VKS provides the ability to manage NLB/ALB through YAML
of Kubernetes, making it easy for you expose Service in Kubernetes to the
outside.
• Enhance security: VKS allows you to create a Private Node Group with only
Private IP and control access to the cluster through the IP Whitelist feature,
ensuring the safety of your system

In [24]:
len(results)

4

# Return scores

In [28]:
results = vector_store.similarity_search_with_score(
    "How to create a pvc of 20Gi in VKS?"
)
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.6691585187705095

page_content='22
Getting Started with VKS' metadata={'source': './../../../data/sample.pdf', 'page': 21, 'start_index': 0}


In [29]:
for res in results:
    print(res, end=f"\n{'_'*100}\n")

(Document(id='16af7257-c570-4599-8d11-25c8ad8c3e72', metadata={'source': './../../../data/sample.pdf', 'page': 21, 'start_index': 0}, page_content='22\nGetting Started with VKS'), 0.6691585187705095)
____________________________________________________________________________________________________
(Document(id='b011f88d-0818-4c6e-80e3-93fd3728baca', metadata={'source': './../../../data/sample.pdf', 'page': 281, 'start_index': 0}, page_content='282\n• Run the following command to deploy a Pod using a Persistent Volume\nCopy\napiVersion: storage.k8s.io/v1\nkind: StorageClass\nmetadata:\n  name: my-expansion-storage-class                    # [1] The Storag\nprovisioner: bs.csi.vngcloud.vn                       # The VNG-CLOUD \nparameters:\n  type: vtype-61c3fc5b-f4e9-45b4-8957-8aa7b6029018    # The volume typ\nallowVolumeExpansion: true                            # MUST set this \n---\napiVersion: v1\nkind: PersistentVolumeClaim\nmetadata:\n  name: my-expansion-pvc                    

# Return documents based on similarity to an embedded query

In [30]:
embedding = embeddings.embed_query("How to create a pvc of 20Gi in VKS?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='22
Getting Started with VKS' metadata={'source': './../../../data/sample.pdf', 'page': 21, 'start_index': 0}


In [31]:
for res in results:
    print(res, end=f"\n{'_'*100}\n")

page_content='22
Getting Started with VKS' metadata={'source': './../../../data/sample.pdf', 'page': 21, 'start_index': 0}
____________________________________________________________________________________________________
page_content='282
• Run the following command to deploy a Pod using a Persistent Volume
Copy
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: my-expansion-storage-class                    # [1] The Storag
provisioner: bs.csi.vngcloud.vn                       # The VNG-CLOUD 
parameters:
  type: vtype-61c3fc5b-f4e9-45b4-8957-8aa7b6029018    # The volume typ
allowVolumeExpansion: true                            # MUST set this 
---
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: my-expansion-pvc                           # [2] The PVC name,
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 20Gi                                # [3] The PVC size,
  storageClassName: my-expansion-storage-class     # [4] The 

# Retrievers

In [35]:
@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)

In [36]:
retriever.batch(
    [
        "Compare VKS private and public clusters",
        "What is the latest release note of VKS?",
    ],
)

[[Document(id='4071a435-a00e-4306-a0d9-3ac7f1c6b214', metadata={'source': './../../../data/sample.pdf', 'page': 163, 'start_index': 0}, page_content='164\nWhen you create a Public Cluster with Public/Private Node Group , the VKS\nsystem will:\n• To enhance the security of your cluster, we have introduced the private cluster\nmodel. The Private Cluster feature helps make your K8S cluster as secure as\npossible, all connections are completely private from the connection between\nnodes to the control plane, the connection from the client to the control plane,\nor the connection from nodes to products. Other services in VNG Cloud such\nas: vStorage, vCR, vMonitor, VNGCloud APIs,...Private Cluster is the ideal\nchoice for services that require strict access control, ensuring compliance\nwith security regulations and data privacy.\nBelow is a comparison table between creating and using Public Cluster and Private\nCluster on the VKS system:\nCriteria Public Cluster Private Cluster\nConnect Us

In [37]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

In [38]:
retriever.batch(
    [
        "Does VKS support GPU?",
        "Does VKS support CSI?",
    ],
)

[[Document(id='01374ba9-b3a6-4a8f-afea-90f600d6b6d2', metadata={'source': './../../../data/sample.pdf', 'page': 145, 'start_index': 0}, page_content='146\n• VKS uses the built-in timesharing ability provided by the NVIDIA GPU and the\nsoftware stack. Starting with the Pascal architecture, NVIDIA GPUs support\ninstruction level preemption. When doing context switching between processes\nrunning on a GPU, instruction-level preemption ensures every process gets a\nfair timeslice. GPU time-sharing provides software-level isolation between the\nworkloads in terms of address space isolation, performance isolation, and error\nisolation.\nfully\nsatur\nate\nthe\nwhole\nGPU.\n• Workl\noads\nthat\nuse\nCUDA\nMPS\nneed\nto\ntolera\nte the\nmem\nory\nprote\nction\nand\nerror\ncontai\nnmen\nt\nlimitat\nions.\nGPU time-slicing\nConfigure GPU time-slicing')],
 [Document(id='b1b017af-1322-44db-a0a6-549cc0a5e2d7', metadata={'source': './../../../data/sample.pdf', 'page': 3, 'start_index': 0}, page_cont